In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/17 02:49:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/17 02:49:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.createOrReplaceTempView('green_data')

In [17]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour,  
    SUM(total_amount) AS amount,
    count(1) as number_records
FROM
    green_data
WHERE 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [18]:
df_green_revenue.coalesce(1).write.parquet('data/report/revenue/green',mode='overwrite')

In [19]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [20]:
df_yellow.createOrReplaceTempView('yellow_data')

In [21]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour,  
    SUM(total_amount) AS amount,
    count(1) as number_records
FROM
    yellow_data
WHERE 
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [24]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow',mode='overwrite')

In [23]:
df_yellow_revenue.show()

+----+-------------------+------------------+--------------+
|zone|               hour|            amount|number_records|
+----+-------------------+------------------+--------------+
|   7|2021-01-21 09:00:00| 5096.399999999999|           240|
| 263|2021-01-05 13:00:00|29519.280000000035|          2208|
| 142|2021-01-05 18:00:00|27924.480000000036|          1752|
| 100|2021-01-19 07:00:00|23697.360000000004|          1728|
| 116|2021-01-29 10:00:00| 4950.959999999999|           216|
| 231|2021-01-29 16:00:00|           17413.2|           816|
| 142|2021-01-24 21:00:00| 7836.239999999998|           552|
| 249|2021-01-23 06:00:00|3907.6800000000007|           192|
| 163|2021-01-12 18:00:00|  49616.6400000001|          3312|
|  75|2021-01-03 14:00:00| 9442.319999999989|           888|
| 140|2021-01-27 20:00:00|26132.160000000014|          1656|
| 151|2021-01-13 18:00:00| 9617.519999999991|           744|
| 263|2021-01-17 22:00:00| 9119.759999999995|           600|
| 107|2021-01-18 16:00:0

In [25]:
df_join = df_green_revenue.join(df_yellow_revenue, on=['hour','zone'],how='outer')

In [26]:
df_join.show()

+-------------------+----+------------------+--------------+--------------------+--------------+
|               hour|zone|            amount|number_records|              amount|number_records|
+-------------------+----+------------------+--------------+--------------------+--------------+
|2021-01-01 10:00:00|   1|              null|          null|   29.75999999999999|            96|
|2021-01-05 16:00:00|   1|              null|          null|  2335.1999999999994|            24|
|2021-01-09 13:00:00|   1|              null|          null|              3055.2|            24|
|2021-01-12 10:00:00|   1|              null|          null|  2047.1999999999998|            24|
|2021-01-12 12:00:00|   1|              null|          null|             2312.64|            24|
|2021-01-19 06:00:00|   1|              null|          null|  1003.9199999999998|            24|
|2021-01-28 14:00:00|   1|              null|          null|  2000.6399999999999|            24|
|2021-01-01 01:00:00|   3|    

In [31]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [32]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [33]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [34]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [30]:
df_join.show()

+-------------------+----+------------------+--------------------+--------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|       yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+--------------------+---------------------+
|2021-01-01 10:00:00|   1|              null|                null|   29.75999999999999|                   96|
|2021-01-05 16:00:00|   1|              null|                null|  2335.1999999999994|                   24|
|2021-01-09 13:00:00|   1|              null|                null|              3055.2|                   24|
|2021-01-12 10:00:00|   1|              null|                null|  2047.1999999999998|                   24|
|2021-01-12 12:00:00|   1|              null|                null|             2312.64|                   24|
|2021-01-19 06:00:00|   1|              null|                null|  1003.9199999999998|                   24|
|2021-01-2

In [35]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2024-02-17 04:27:40--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.42.80, 52.216.32.120, 52.216.211.168, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.42.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi+_zone_lookup.csv’

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0s      

2024-02-17 04:27:41 (60.4 MB/s) - ‘taxi+_zone_lookup.csv’ saved [12322/12322]



In [36]:
!head taxi+_zone_lookup.csv

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"


In [37]:
df = spark.read \
    .option("header", "true") \
    .csv('taxi+_zone_lookup.csv')

In [38]:
df.write.parquet('zones')

In [39]:
df_zones = spark.read.parquet('zones/')

In [40]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [41]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')